## 트랜스포머 어텐션

어텐션(attention)은 주로 언어 번역에서 사용되기 때문에 `인코더`와 `디코더` 네트워크를 사용합니다. 즉, 입력에 대한 벡터 변환을 인코더(encoder)에서 처리하고 모든 벡터를 디코더로 보냅니다. 이렇게 모든 벡터를 전달하는 이유는 시간이 흐를수록 초기 정보를 잃어버리는 `기울기 소멸 문제를 해결`하기 위해서입니다. 하지만 모든 벡터가 전달되기 때문에 행렬 크기가 굉장히 커지는 단점이 있는데, 이것을 해결하기 위해 소프트맥스 함수를 사용하여 가중합을 구하고 그 값을 디코더에 전달합니다.

가중합만 전달되었더라도 정보를 많이 전달받은 디코더는 부담일 수밖에 없습니다. 따라서 디코더는 은닉 상태에 대해 중점적으로 집중(attention)해서 보아야 할 벡터를 소프트맥스 함수로 점수를 매긴 후 각각을 은닉 상태의 벡터들과 곱합니다. 그리고 이 은닉 상태를 모두 더해서 하나의 값으로 만듭니다. 즉, 어텐션은 `모든 벡터 중에서 꼭 살펴보아야 할 벡터들을 집중`하겠다는 의미입니다.

![](../Static/564.jpg)

트랜스포머(transformer)는 2017년 6월에 'Attention is All You Need' 논문에서 발표된 것으로 어텐션을 극대화하는 방법입니다. 어텐션에서 다룬 인코더와 디코더에는 네트워크가 하나씩 있었습니다. 하지만 트랜스포머는 인코더와 디코더를 여러 개 중첩시킨 구조입니다. 이때 각각의 인코더와 디코더를 블록(block)이라고 합니다.

![](../Static/565_1.jpg)

### 인코더 블록

먼저 인코더 블록 구조를 살펴보겠습니다. 참고로 모든 블록 구조는 동일합니다. 하나의 인코더는 `셀프 어텐션(self attention)`과 `전방향 신경망(feed forward neural network)`으로 구성되어 있습니다. 인코더에서는 단어를 벡터로 임베딩하며, 이를 셀프 어텐션과 전방향 신경망으로 전달합니다. 이때 셀프 어텐션은 문장에서 각 단어끼리 얼마나 관계가 있는지를 계산해서 반영합니다. 즉, 셀프 어텐션으로 `문장 안에서 단어 간 관계를 파악`할 수 있습니다. 셀프 어텐션에서 파악된 단어 간 관계는 전방향 신경망으로 전달됩니다.

![](../Static/565_2.jpg)

### 디코더 블록

이번에는 디코더를 알아보겠습니다. 디코더는 층을 총 세 개 가지고 있는데, 인코더에서 넘어온 벡터가 처음으로 만나는 것이 `셀프 어텐션 층`입니다. 즉, 인코더와 동일하다고 이해하면 됩니다. 셀프 어텐션 층을 지나면 `인코더-디코더 어텐션(encoder-decoder attention) 층`이 있습니다. 인코더-디코더 어텐션 층에서는 인코더가 처리한 정보를 받아 어텐션 매커니즘을 수행하고, 마지막으로 전방향 신경망으로 데이터가 전달됩니다.

![](../Static/566.jpg)



어텐션 매커니즘을 구체적으로 수식을 이용하여 알아보겠습니다.

어텐션 매커니즘을 이용하기 위해서는 가장 먼저 `어텐션 스코어`를 구해야 합니다.

$$e_{ij} = a(s_{i-1}, h_j)$$

어텐션 스코어란 현재 디코더의 시점 $i$에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 값($h_j$)이 디코더의 현 시점의 은닉 상태($s_i$)와 얼마나 관련이 있는지(유사한지)를 판단하는 값입니다. 따라서 어텐션 스코어는 앞의 수식처럼 인코더의 모든 은닉 상태의 값($h_j$)과 디코더에서의 이전 시점의 은닉 상태($s_{i-1}$) 값을 이용하여 구할 수 있습니다.

어텐션 스코어가 계산되었다면 이 값을 `소프트맥스(softmax)` 함수에 적용하여 확률로 변환합니다. 이렇게 계산된 0~1 사이의 값들이 특정 시점(time step)에 대한 가중치, 즉 시간의 가중치가 되며 다음과 같은 수식을 이용합니다.

![](../Static/fn2-73.jpg)

이제 시간의 가중치($a_{ij}$)와 은닉 상태($h_j$)의 가중합을 계산하면 하나의 벡터가 계산되는데, 이 것이 `컨텍스트 벡터(context vector)`입니다.

마지막으로 디코더의 은닉 상태를 구해야 합니다. 디코더의 은닉 상태를 구하기 위해서는 컨텍스트 벡터와 디코더 이전 시점의 은닉 상태와 출력이 필요합니다.

'They are cats' 라는 입력 시퀀스에 대해 현 시점의 디코더 은닉 상태인 $s_2$와 그 출력인 $y_2$를 계산하기 위해 먼저 $s_1$과 모든 인코더 은닉 상태에 대한 어텐션 스코어를 계산합니다. $e_{21}$, $e_{22}$, $e_{23}$ 처럼 어텐션 스코어를 구했으면 소프트맥스 함수를 적용하여 시간의 가중치($a_{21}$, $a_{22}$, $a_{23}$)를 구합니다. 이후 시간의 가중치와 인코더의 은닉 상태 값들을 이용하여 가중합을 계산함으로써 컨텍스트 벡터($c_2$)를 구합니다.

최종적으로 앞에서 구한 컨텍스트 벡터와 디코더 이전 시점의 은닉 상태와 출력을 이용하여 최종적으로 다음 디코더의 은닉 상태($y_2$)를 출력합니다.

![](../Static/567.jpg)

자세한 구현 방법은 이어서 seq2seq, 버트(BERT)에서 예제로 알아보겠습니다.